# OLS and ML estimations with Julia JuMP

This notebook showcases how to estimate parameters based on OLS or ML methods using [Julia's JuMP package](https://jump.dev/JuMP.jl/stable/).
The main purpose of this document is not to argue that JuMP is a perfect tool for simple estimations but rather to show how to use JuMP with data.

In [1]:
# using Pkg

# Pkg.add("JuMP")
# Pkg.add("DataFrames")
# Pkg.add("Ipopt")
# Pkg.add("BenchmarkTools")
# Pkg.add("Optim")
# Pkg.add("ForwardDiff")

In [2]:
using JuMP, Ipopt
using Random
using BenchmarkTools
using Optim
# using DataFrames
# using ForwardDiff

## Data generation

I consider the following equation for estimation:
$$
    y_i = \alpha + X_i' \beta + u_i.
$$
The error term follows the standard normal distribution.
The data are simulated with $X_i$ and all coefficients following the standard normal distribution.

In [3]:
Random.seed!(3);

In [4]:
N = 1_000;
K = 3;

In [5]:
β = randn(Float64, K + 1);

In [6]:
X = hcat(ones(N), randn(Float64, (N, K)));
u = randn(Float64, N);
y = X * β .+ u;

# OLS

## OLS with matrix inversion

In [7]:
function estimate_ols(y, X)
    βhat = (X \ X) * (X \ y);
    return βhat
end

estimate_ols (generic function with 1 method)

In [8]:
βhat_ols = estimate_ols(y, X);

In [9]:
@benchmark estimate_ols(y, X)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  39.458 μs …  11.431 ms  ┊ GC (min … max): 0.00% … 98.41%
 Time  (median):     41.625 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   55.592 μs ± 151.815 μs  ┊ GC (mean ± σ):  9.66% ±  7.36%

  █▄▄▃▂▃▂▂▁▁▁▁                                                 ▁
  ████████████████▇▆▆▆▅▅▃▅▅▃▃▅▅▄▅▅▄▅▃▄▄▄▄▁▄▄▄▃▃▁▅▄▃▄▁▁▁▃▃▁▄▁▃▃ █
  39.5 μs       Histogram: log(frequency) by time       306 μs <

 Memory estimate: 172.95 KiB, allocs estimate: 55.

## OLS with JuMP

Here I use JuMP for OLS estimation.
I use the `Ipopt` solver.

In [10]:
function estimate_ols_jump(y, X; K = K, N = N)

    model = Model(Ipopt.Optimizer)
    set_silent(model)
    set_string_names_on_creation(model, false)
    @variable(model, β[1:(K + 1)])
    @objective(
        model,
        Min,
        sum((y[i] - sum(X[i, j] * β[j] for j in 1:(K + 1)))^2 for i in 1:N)
    )
    optimize!(model)
    @assert is_solved_and_feasible(model)

    return value.(β)
end

estimate_ols_jump (generic function with 1 method)

In [11]:
βhat_ols_jump = estimate_ols_jump(y, X);


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************



In [12]:
@benchmark estimate_ols_jump(y, X)

BenchmarkTools.Trial: 895 samples with 1 evaluation.
 Range (min … max):  3.728 ms … 37.804 ms  ┊ GC (min … max): 0.00% … 68.12%
 Time  (median):     4.758 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   5.577 ms ±  2.406 ms  ┊ GC (mean ± σ):  5.66% ±  8.31%

    ▃▆█▆▃▁                                                    
  ▂▆██████▇▄▅▃▄▄▄▃▃▂▂▂▃▂▂▃▃▃▃▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▁▂▁▂▂▁▁▂▁▂▁▂▂▂▁▂ ▃
  3.73 ms        Histogram: frequency by time        13.6 ms <

 Memory estimate: 3.90 MiB, allocs estimate: 36453.

#### Performance comparisons: Access global variables

As [in the general Julia performance tips](https://docs.julialang.org/en/v1/manual/performance-tips/index.html#Avoid-untyped-global-variables), it is a good practice to avoid to call untyped global variables in functions.
In `estimate_ols_jump`, I added `K` and `N` as keyword arguments of the function.
Not doing this (= accessing the untyped global variables from the function) can increase the allocated memory and the runtime:

In [13]:
function estimate_ols_jump_global(y, X)

    model = Model(Ipopt.Optimizer)
    set_silent(model)
    @variable(model, β[1:(K + 1)])
    @objective(
        model,
        Min,
        sum((y[i] - sum(X[i, j] * β[j] for j in 1:(K + 1)))^2 for i in 1:N)
    )
    optimize!(model)
    @assert is_solved_and_feasible(model)

    return value.(β)
end

estimate_ols_jump_global (generic function with 1 method)

In [14]:
@benchmark estimate_ols_jump_global(y, X)

BenchmarkTools.Trial: 818 samples with 1 evaluation.
 Range (min … max):  4.208 ms … 36.129 ms  ┊ GC (min … max): 0.00% … 59.37%
 Time  (median):     5.259 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   6.106 ms ±  2.598 ms  ┊ GC (mean ± σ):  5.70% ±  8.52%

   ▄███▆▅▃▂▂         ▃▂▁▁                                     
  ██████████▇██▇▇▆▄▆▆██████▆▇▆▅▆▅▁▅▄▁▁▄▄▁▄▁▄▁▁▁▁▁▁▁▁▄▄▄▁▁▁▁▄ █
  4.21 ms      Histogram: log(frequency) by time     17.7 ms <

 Memory estimate: 4.30 MiB, allocs estimate: 52459.

## Estimates comparison

In [34]:
[βhat_ols βhat_ols_jump β]

4×3 Matrix{Float64}:
 -0.627644  -0.627644  -0.663764
  1.36376    1.36376    1.36321
  0.756496   0.756496   0.717411
  0.660766   0.660766   0.678743

# Maximum Likelihood

Next, I estimate the parameters with the maximum likelihood (ML) method.
Here, I also estimate the standard deviation of the error term (the true s.d. is 1).

In [16]:
initial_x = ones(K + 2);

In [17]:
function calculate_log_likelihood(param, y, X; K = K, N = N)
    β = @view param[1:(K + 1)]
    σ = param[K + 2]
    log_likelihood = (
        N / 2 * log(1 / (2 * π * σ^2)) -
        sum((y - X * β).^2) / (2 * σ^2)
    )

    return log_likelihood
end

calculate_log_likelihood (generic function with 1 method)

## With `Optim`

First, I do the ML estimation with Julia's `Optim` package, which is one of the most popular optimization package.

In [18]:
function estimate_ml_optim(y, X, initial_x)

    ml_res_optim = optimize(
        param -> - calculate_log_likelihood(param, y, X), initial_x
    )
    
    return ml_res_optim.minimizer
end

estimate_ml_optim (generic function with 1 method)

In [19]:
est_ml_optim = estimate_ml_optim(y, X, initial_x);
βhat_ml_optim = est_ml_optim[1:(K + 1)];
σhat_ml_optim = est_ml_optim[K + 2];

In [20]:
@benchmark estimate_ml_optim(y, X, initial_x)

BenchmarkTools.Trial: 3807 samples with 1 evaluation.
 Range (min … max):  751.208 μs … 25.035 ms  ┊ GC (min … max):  0.00% …  0.00%
 Time  (median):     829.042 μs              ┊ GC (median):     0.00%
 Time  (mean ± σ):     1.306 ms ±  1.174 ms  ┊ GC (mean ± σ):  25.41% ± 23.56%

  █▇▄▄▄▁                ▂▂▃▄▅▄▃▂                               ▁
  ██████▇▇▆▃▆▇▄▆▅▃▃▃▆▅▅▆█████████▇▆▇▆▅▆▅▄▅▅▁▃▃▅▅▄▁▁▅▄▃▃▃▁▄▃▄▁▄ █
  751 μs        Histogram: log(frequency) by time      3.96 ms <

 Memory estimate: 8.19 MiB, allocs estimate: 4577.

#### Performance comparison: Copy param, not view

In `calculate_log_likelihood`, $\beta$ is extracted from the parameter input with `@view` macro.
Without `@view`, Julia creates a copy when making the $\beta$ variable, which results in additional memory allocations.

In [21]:
function calculate_log_likelihood_no_view(param, y, X; K = K, N = N)
    β = param[1:(K + 1)] # <======== Not using @view
    σ = param[K + 2]
    log_likelihood = (
        N / 2 * log(1 / (2 * π * σ^2)) -
        sum((y - X * β).^2) / (2 * σ^2)
    )

    return log_likelihood
end

function estimate_ml_optim_no_view(y, X, initial_x)

    ml_res_optim = optimize(
        param -> - calculate_log_likelihood_no_view(param, y, X), initial_x
    )
    
    return ml_res_optim.minimizer
end

estimate_ml_optim_no_view (generic function with 1 method)

In [22]:
@benchmark estimate_ml_optim_no_view(y, X, initial_x)

BenchmarkTools.Trial: 3741 samples with 1 evaluation.
 Range (min … max):  767.041 μs …  18.438 ms  ┊ GC (min … max):  0.00% … 64.76%
 Time  (median):     874.417 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):     1.328 ms ± 941.587 μs  ┊ GC (mean ± σ):  25.65% ± 23.51%

  ▇█▅▄▄▂                  ▁▂▄▄▄▃▂▁                              ▁
  ███████▇▇▅▆▅▆▅▅▅▅▆▃▅▃▆▇██████████▇▇▆▆▅▃▅▅▄▄▃▄▄▂▃▃▄▄▂▃▂▃▃▂▂▂▂▃ █
  767 μs        Histogram: log(frequency) by time        3.9 ms <

 Memory estimate: 8.22 MiB, allocs estimate: 5285.

## With `Optim` with a constraint

In the optimization above, there was no constraint for $\sigma$, which could have returned an error due to negative standard deviations.
To prevent this, I add a positive constraint for $\sigma$ to the model.

In [23]:
function estimate_ml_optim_constrainted(y, X, initial_x)

    ml_res_optim = optimize(
        param -> - calculate_log_likelihood(param, y, X), 
        vcat(repeat([-Inf], K + 1), [1e-10]),
        repeat([Inf], K + 2),
        initial_x,
        Fminbox(GradientDescent())
    )
    
    return ml_res_optim.minimizer
end

estimate_ml_optim_constrainted (generic function with 1 method)

In [24]:
est_ml_optim_constrained = estimate_ml_optim_constrainted(y, X, initial_x);
βhat_ml_optim_constrained = est_ml_optim[1:(K + 1)];
σhat_ml_optim_constrained = est_ml_optim[K + 2];

In [25]:
@benchmark estimate_ml_optim_constrainted(y, X, initial_x)

BenchmarkTools.Trial: 942 samples with 1 evaluation.
 Range (min … max):  3.310 ms … 32.614 ms  ┊ GC (min … max):  0.00% … 0.00%
 Time  (median):     4.863 ms              ┊ GC (median):    24.65%
 Time  (mean ± σ):   5.296 ms ±  1.766 ms  ┊ GC (mean ± σ):  25.92% ± 6.06%

          ██▅▂▁   ▃▃▂                                         
  █▇▁▄▁▁▁▁██████▆▇███▇▇▆▅▆▄▅▄▄▅▅▅▄▁▁▄▁▄▁▄▁▁▁▁▄▁▄▁▁▁▁▁▁▁▄▁▁▄▄ █
  3.31 ms      Histogram: log(frequency) by time     12.9 ms <

 Memory estimate: 33.90 MiB, allocs estimate: 19486.

## With JuMP

Here, I estimate the parameters with ML with JuMP.
As shown below, the run time is a bit slower but pretty comparable to the `Optim` implementation with constraints.

### Directly specify the ML objective function within JuMP

First, I directly calculate the objective function within JuMP.
This part uses [the code in the official JuMP document](calculate_log_likelihood_no_view) with simplifications.

In [26]:
function estimate_ml_jump_direct(y, X; K = K, N = N)
        
    model = Model(Ipopt.Optimizer)
    set_silent(model)
    @variable(model, β[1:(K + 1)])
    @variable(model, σ >= 0.0)
    @objective(
        model,
        Max,
        N / 2 * log(1 / (2 * π * σ^2)) -
        sum((y[i] - sum(X[i, j] * β[j] for j in 1:(K + 1)))^2 for i in 1:N) / (2 * σ^2)
    )
    optimize!(model)
    @assert is_solved_and_feasible(model)

    return vcat(value.(β), value(σ))

end

estimate_ml_jump_direct (generic function with 1 method)

In [27]:
est_ml_jump_direct = estimate_ml_jump_direct(y, X);
βhat_ml_jump_direct = est_ml_jump_direct[1:(K + 1)];
σhat_ml_jump_direct = est_ml_jump_direct[K + 2];

In [28]:
@benchmark estimate_ml_jump_direct(y, X)

BenchmarkTools.Trial: 626 samples with 1 evaluation.
 Range (min … max):  5.867 ms … 41.218 ms  ┊ GC (min … max): 0.00% … 49.84%
 Time  (median):     7.011 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   7.980 ms ±  2.892 ms  ┊ GC (mean ± σ):  4.37% ±  7.41%

    ▄██▄▁                                                     
  ▄██████▄▅▅▅▄▄▃▄▃▂▃▃▃▂▃▂▃▃▃▃▄▃▂▃▂▂▂▂▁▁▂▂▂▁▂▂▂▂▂▁▁▁▁▁▁▁▂▂▁▁▂ ▃
  5.87 ms        Histogram: frequency by time        18.5 ms <

 Memory estimate: 3.99 MiB, allocs estimate: 38468.

### Call the function to calculate log likelihood

Here, I create a function to calculate log-likelihood and call it within JuMP.
In JuMP, this kind of function called from JuMP models is called "user-defined function".
Since user-defined functions do not accept vector inputs ([document](https://jump.dev/JuMP.jl/stable/manual/nonlinear/#User-defined-operators-with-vector-inputs)), I have to use Julia's splatting syntax.
Interestingly, using a user-defined function does not increase the run-time, as shown below.

In [29]:
function calculate_log_likelihood_jump(args...; K = K, N = N)

    param = args[1]
    y = args[2]
    X = args[3]

    β = @view param[1:(K + 1)]
    σ = param[K + 2]
    log_likelihood = (
        N / 2 * log(1 / (2 * π * σ^2)) -
        sum((y - X * β).^2) / (2 * σ^2)
    )

    return log_likelihood
end

calculate_log_likelihood_jump (generic function with 1 method)

In [30]:
function estimate_ml_jump_call_ll(y, X; K = K, N = N)
        
    model = Model(Ipopt.Optimizer)
    set_silent(model)
    @variable(model, param[1:(K + 2)])
    set_lower_bound(param[K + 2], 0)
    @objective(
        model,
        Max,
        calculate_log_likelihood_jump(param, y, X)
    )
    optimize!(model)
    @assert is_solved_and_feasible(model)

    return value.(param)

end

estimate_ml_jump_call_ll (generic function with 1 method)

In [31]:
est_ml_jump_call_ll = estimate_ml_jump_call_ll(y, X);
βhat_ml_jump_call_ll = est_ml_jump_call_ll[1:(K + 1)];
σhat_ml_jump_call_ll = est_ml_jump_call_ll[K + 2];

In [32]:
@benchmark estimate_ml_jump_call_ll(y, X)

BenchmarkTools.Trial: 623 samples with 1 evaluation.
 Range (min … max):  5.830 ms … 76.622 ms  ┊ GC (min … max): 0.00% … 74.09%
 Time  (median):     6.945 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   8.020 ms ±  3.937 ms  ┊ GC (mean ± σ):  4.88% ±  7.58%

    ███▅▁                                                     
  ▅██████▆▆▅▄▄▃▃▄▃▄▃▃▃▃▃▂▂▂▁▁▂▂▃▃▃▃▃▃▃▂▂▂▃▂▁▁▂▁▂▁▁▂▁▁▂▂▁▁▁▁▃ ▃
  5.83 ms        Histogram: frequency by time        18.2 ms <

 Memory estimate: 4.01 MiB, allocs estimate: 38495.

## Estimates comparison

In [33]:
[βhat_ml_optim βhat_ml_optim_constrained βhat_ml_jump_direct βhat_ml_jump_call_ll β]

4×5 Matrix{Float64}:
 -0.627644  -0.627644  -0.627644  -0.627644  -0.663764
  1.36376    1.36376    1.36376    1.36376    1.36321
  0.756495   0.756495   0.756496   0.756496   0.717411
  0.660767   0.660767   0.660766   0.660766   0.678743